<center>
    <h1>
            Probabilidad y Estadística
    </h1>
    <h2>
            Trabajo Práctico
    </h2>
</center>

<table>
<thead>
  <tr>
    <th>Integrante</th>
    <th>Legajo</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>Cintia Capece</td>
    <td>xxxxx</td>
  </tr>
  <tr>
    <td>Nicolás Bellesi</td>
    <td>xxxxx</td>
  </tr>
  <tr>
    <td>Lucas Nevado</td>
    <td>40768</td>
  </tr>
</tbody>
</table>

## Índice

* [Setup](#setup)
* [Parte 1: Simulación](#parte_1)
    * [1.1 Generador Bernoulli](#p_1_1)
    * [1.2 Generador Binomial](#p_1_2)
    * [1.3 Generador Exponencial](#p_1_3)
    * [1.4 Generador Normal (y Normal Estándar)](#p_1_4)
* [Parte 2: Estadística descriptiva](#parte_2)
    * [2.3 Muestra n=50 Bin(10, 0.3) y función de distribución empírica](#p_2_3)
    * [2.4 Nueva muestra a partir de la función de distribución empírica. Media y varianza muestral](#p_2_4)
    * [2.5 Repetición del experimento](#p_2_5)
* [Bibliografía](#bibliografia)

---

## Setup<a class="anchor" id="setup"></a>

In [16]:
import random
import math
import numpy as np
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import row
from bokeh.models import Span


output_notebook()

def muestra_gen(muestra_tam, distribucion_gen, *distribucion_args):
    """
    muestra_tam: Tamaño de la muestra
    distribucion_gen: Generador de una distribución, ej: exponencial_gen
    distribucion_args: Argumentos necesarios para la distribución, ej: 0.5
    """
    muestra = []
    for i in range(muestra_tam):
        valor = distribucion_gen(*distribucion_args)
        muestra.append(valor)

    return muestra

def crear_histograma(muestra, ancho_de_banda = 0):
    """
    A partir de una muestra, devuelve una histograma de bokeh
    El ancho de banda es opcional
    """
    if ancho_de_banda == 0:
        hist, edges = np.histogram(muestra, density=True)
    else:
        hist, edges = np.histogram(muestra, density=True, bins=math.floor((max(muestra) - min(muestra)) / ancho_de_banda))

    p = figure()
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="white")
    return p

Loading BokehJS ...

## Parte 1: Simulación<a class="anchor" id="parte_1"></a>

### 1) Generador Bernoulli<a class="anchor" id="p_1_1"></a>

In [17]:
def bernoulli_gen(p):
    uniforme = random.uniform(0, 1)
    if uniforme < p:
        return 1
    else:
        return 0

In [18]:
bernoulli_gen(0.75)

1

### 2) Generador Binomial<a class="anchor" id="p_1_2"></a>

In [19]:
def binomial_gen(n, p):
    exitos = 0
    for i in range(n):
        exitos += bernoulli_gen(p)

    return exitos

In [20]:
binomial_gen(10, 0.5)

5

### 3) Generador Exponencial<a class="anchor" id="p_1_3"></a>

In [21]:
def exponencial_gen(λ):
    """
    Utilizamos el método de la función inversa
    """
    uniforme = random.uniform(0, 1)
    
    exponencial = - (1 / λ) * math.log(uniforme)

    return exponencial

In [22]:
exponencial_gen(3)

0.02120838105015424

### 4) Generador Normal (y Normal Estándar)<a class="anchor" id="p_1_4"></a>

In [23]:
def std_normal_gen():
    """
    Utilizamos el método de Box-Muller
    """
    uniforme_U = random.uniform(0, 1)
    uniforme_V = random.uniform(0, 1)

    std_normal = math.sqrt(-2 * math.log(uniforme_U)) * math.cos(2 * math.pi * uniforme_V)
    
    return std_normal

In [24]:
std_normal_gen()

-1.0946960449066336

In [25]:
def normal_gen(μ = 0, σ = 1):
    normal = std_normal_gen() * σ + μ

    return normal

In [26]:
normal_gen(1, 1)

1.1665539665497244

### 3) Muestra n=50 Bin(10, 0.3) y función de distribución empírica<a class="anchor" id="p_2_3"></a>

In [27]:
muestra_bin = muestra_gen(50, binomial_gen, 10, 0.3)

def fde(muestra, x):
    muestra_ordenada = np.sort(muestra)
    N = muestra_ordenada.size
    return np.searchsorted(muestra_ordenada, x) / N

fde(muestra_bin, 3)

0.34

### 4) Nueva muestra a partir de la función de distribución empírica. Media y varianza muestral<a class="anchor" id="p_2_4"></a>

In [28]:
def muestra_fde(muestra):
    muestra_fde = []
    for i in muestra_bin:
        v = fde(muestra_bin, i)
        muestra_fde.append(v)
    return muestra_fde

muestra_fde_bin = muestra_fde(muestra_bin)
    
def media_muestral(muestra):
    return sum(muestra) / len(muestra)

def varianza_muestral(muestra):
    return (sum([x ** 2 for x in muestra]) / len(muestra)) - (media_muestral(muestra) ** 2)

print("Media Muestral Bin(10, 0.3): ", round(media_muestral(muestra_bin), 3))
print("Media Muestral FDE(Bin(10, 0.3)): ", round(media_muestral(muestra_fde_bin), 3))

print("Varianza Muestral Bin(10, 0.3): ", round(varianza_muestral(muestra_bin), 3))
print("Varianza Muestral FDE(Bin(10, 0.3)): ", round(varianza_muestral(muestra_fde_bin), 3))

hist1 = crear_histograma(muestra_bin)
linea_media_muestral_bin = Span(location= media_muestral(muestra_bin), dimension='height', line_color='red', line_width=3)
hist1.renderers.extend([linea_media_muestral_bin])

hist2 = crear_histograma(muestra_fde_bin)
linea_media_muestral_fde = Span(location= media_muestral(muestra_fde_bin), dimension='height', line_color='red', line_width=3)
hist2.renderers.extend([linea_media_muestral_fde])

show(row(hist1, hist2, sizing_mode='scale_both'))

Media Muestral Bin(10, 0.3):  2.8
Media Muestral FDE(Bin(10, 0.3)):  0.373
Varianza Muestral Bin(10, 0.3):  1.48
Varianza Muestral FDE(Bin(10, 0.3)):  0.082


### 5) Repetición del experimento<a class="anchor" id="p_2_5"></a>

In [29]:
# Primera repetición
muestra_bin = muestra_gen(50, binomial_gen, 10, 0.3)
muestra_fde_bin = muestra_fde(muestra_bin)

print("2 - Media Muestral Bin(10, 0.3): ", round(media_muestral(muestra_bin), 3))
print("2 - Media Muestral FDE(Bin(10, 0.3)): ", round(media_muestral(muestra_fde_bin), 3))

print("2 - Varianza Muestral Bin(10, 0.3): ", round(varianza_muestral(muestra_bin), 3))
print("2 - Varianza Muestral FDE(Bin(10, 0.3)): ", round(varianza_muestral(muestra_fde_bin), 3))

hist1 = crear_histograma(muestra_bin)
linea_media_muestral_bin = Span(location= media_muestral(muestra_bin), dimension='height', line_color='red', line_width=3)
hist1.renderers.extend([linea_media_muestral_bin])

hist2 = crear_histograma(muestra_fde_bin)
linea_media_muestral_fde = Span(location= media_muestral(muestra_fde_bin), dimension='height', line_color='red', line_width=3)
hist2.renderers.extend([linea_media_muestral_fde])

show(row(hist1, hist2, sizing_mode='scale_both'))

# Segunda repetición
muestra_bin = muestra_gen(50, binomial_gen, 10, 0.3)
muestra_fde_bin = muestra_fde(muestra_bin)

print("3 - Media Muestral Bin(10, 0.3): ", round(media_muestral(muestra_bin), 3))
print("3 - Media Muestral FDE(Bin(10, 0.3)): ", round(media_muestral(muestra_fde_bin), 3))

print("3 - Varianza Muestral Bin(10, 0.3): ", round(varianza_muestral(muestra_bin), 3))
print("3 - Varianza Muestral FDE(Bin(10, 0.3)): ", round(varianza_muestral(muestra_fde_bin), 3))

hist1 = crear_histograma(muestra_bin)
linea_media_muestral_bin = Span(location= media_muestral(muestra_bin), dimension='height', line_color='red', line_width=3)
hist1.renderers.extend([linea_media_muestral_bin])

hist2 = crear_histograma(muestra_fde_bin)
linea_media_muestral_fde = Span(location= media_muestral(muestra_fde_bin), dimension='height', line_color='red', line_width=3)
hist2.renderers.extend([linea_media_muestral_fde])

show(row(hist1, hist2, sizing_mode='scale_both'))

2 - Media Muestral Bin(10, 0.3):  3.02
2 - Media Muestral FDE(Bin(10, 0.3)):  0.391
2 - Varianza Muestral Bin(10, 0.3):  1.58
2 - Varianza Muestral FDE(Bin(10, 0.3)):  0.076


3 - Media Muestral Bin(10, 0.3):  2.9
3 - Media Muestral FDE(Bin(10, 0.3)):  0.399
3 - Varianza Muestral Bin(10, 0.3):  2.25
3 - Varianza Muestral FDE(Bin(10, 0.3)):  0.081


## Prueba histograma

In [30]:
muestra_normal_estandar_1000 = muestra_gen(1000, std_normal_gen)
histograma = crear_histograma(muestra_normal_estandar_1000)
show(histograma)

# También se puede mostrar en una fila varios histogramas
histograma_con_ancho = crear_histograma(muestra_normal_estandar_1000, 0.5)
show(row(histograma, histograma_con_ancho, sizing_mode='scale_both'))

## Bibliografía<a class="anchor" id="bibliografia"></a>